### Load data and get first impression

In [1]:
import pandas as pd
import numpy as np

In [2]:
column_headers = pd.read_csv('hit_data.processed.merged_column_headers.tsv', sep='\t')

In [3]:
%%time

df = pd.read_csv('hit_data.sample_160509-160511.tsv.gz', compression='gzip', sep='\t', encoding='iso-8859-1', quoting=3, low_memory=False, names=column_headers)

C:\Users\alex.merdian-tarko\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\io\parsers.py:678: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)


Wall time: 3min 32s


In [4]:
df.shape

(1198578, 138)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1198578 entries, 0 to 1198577
Columns: 138 entries, accept_language to product_item_price
dtypes: float64(8), int64(55), object(75)
memory usage: 1.2+ GB


In [6]:
pd.set_option('display.max_columns', df.shape[1])
df.head()

,accept_language,browser,c_color,carrier,click_action,click_context,click_context_type,click_tag,code_ver,color,connection_type,country,ct_connect_type,curr_factor,curr_rate,daily_visitor,date_time,domain,duplicate_events,duplicate_purchase,exclude_hit,first_hit_page_url,first_hit_pagename,first_hit_referrer,first_hit_time_gmt,geo_city,geo_country,geo_dma,geo_region,geo_zip,hit_source,hit_time_gmt,hitid_high,hitid_low,homepage,hourly_visitor,j_jscript,javascript,language,last_purchase_num,monthly_visitor,new_visit,os,page_url,paid_search,post_browser_height,post_browser_width,post_campaign,post_channel,post_cookies,post_currency,post_evar1,post_evar2,post_evar3,post_evar4,post_evar10,post_evar17,post_evar18,post_evar22,post_evar26,post_evar34,post_evar35,post_evar36,post_evar37,post_evar38,post_evar50,post_evar60,post_evar61,post_evar62,post_event_list,post_java_enabled,post_keywords,post_mvvar1,post_page_event,post_page_event_var2,post_pagename,post_pagename_no_url,post_persistent_cookie,post_prop13,post_prop14,post_prop15,post_purchaseid,post_referrer,post_search_engine,post_tnt,post_tnt_action,post_transactionid,post_visid_type,post_zip,quarterly_visitor,ref_domain,ref_type,resolution,s_resolution,sampled_hit,search_page_num,secondary_hit,service,sourceid,stats_server,truncated_hit,user_agent,user_hash,user_server,userid,username,va_closer_detail,va_closer_id,va_finder_detail,va_finder_id,va_instance_event,va_new_engagement,visid_new,visit_keywords,visit_num,visit_page_num,visit_referrer,visit_search_engine,visit_start_page_url,visit_start_pagename,visit_start_time_gmt,weekly_visitor,yearly_visitor,post_evar40,purchase_boolean,product_view_boolean,cart_open_boolean,checkout_boolean,cart_addition_boolean,cart_removal_boolean,cart_view_boolean,campaign_view_boolean,page_view_boolean,visitor_id,product_categories,product_items,product_item_number,product_item_price
0,"de-DE,de,en-US,en",2661327960,32.0,swisscom.ch:swisscom ag,NaN,NaN,0,NaN,JS-1.5.2,0,4,215,NaN,2,1.0,1,2016-05-09 00:01:05,swisscom.ch,NaN,0,0,https://ecommerce-shop.de/search?brand_name=An...,/search?brand_name=anastasia%20beverly%20hills...,NaN,1462744865,zurich,che,0,zh,8045,1,1462744865,3141220678870532096,6446378417788105690,U,1,1.6,0,60,0,1,1,3716544477,https://ecommerce-shop.de/search?brand_name=An...,0,0,0,Paid Search - AdWords:ci:353946413,Search Results,Y,CHF,NaN,NaN,NaN,NaN,NaN,Suchergebnisse,Search Results,/search,NaN,NaN,572fb71b475f08.09304145,NaN,n,NaN,NaN,NaN,NaN,NaN,"200,201,269,20,110,116,119,120,121,122,126,130...",U,NaN,NaN,0,NaN,/search?brand_name=anastasia%20beverly%20hills...,/search?brand_name=anastasia%20beverly%20hills...,Y,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,5,::hash::0,1,NaN,6,0,375x667,Y,0,0,ss,0,www34.ams1.omniture.com,N,Mozilla/5.0 (iPhone; CPU iPhone OS 9_3_1 like ...,1841006947,NaN,100030943,ecommerce-shoplive,Paid Search - AdWords:ci:353946413,10,Paid Search - AdWords:ci:353946413,10,1,1,N,NaN,1,1,NaN,0,https://ecommerce-shop.de/search?brand_name=An...,/search?brand_name=anastasia%20beverly%20hills...,1462744865,1,1,NaN,0,0,0,0,0,0,0,1,1,469902274971074530_2331856232162590288,SchÃ¶nheit & Gesundheit / Make-up & Schminke /...,214498-1;224512-1;228076-1;116246-1;329229-1;1...,;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;,36.80;25.60;25.60;28.80;32.00;35.20;28.80;34.0...
1,"en-US,en;q=0.8",2734330576,24.0,NaN,NaN,NaN,0,NaN,JS-1.5.2,2,2,215,NaN,2,1.0,1,2016-05-09 00:01:05,swisscom.ch,NaN,0,0,https://ecommerce-shop.de/search?brand_name=Gu...,/search?brand_name=guam|guam,https://www.google.pl/,1462744865,mannedorf,che,0,zh,8708,1,1462744865,3141220678870532096,4723997855218510157,U,1,1.6,7,45,0,1,1,1240087047,https://ecommerce-shop.de/search?brand_name=Gu...,1,799,1600,Paid Search - AdWords:ci:353951213,Search Results,Y,CHF,NaN,NaN,NaN,NaN,NaN,Suchergebnisse,Search Results,/search,https://www.google.pl/,NaN,572fb71a0d1d57.58871989,NaN,n,NaN,NaN,NaN,NaN,NaN,"200,201,269,20,110,116,119,120,121,122,125,126...",N,::empty::,NaN,0,NaN,/sear

### Make sure date range of sample is correct

In [7]:
df.date_time.min()

'2016-05-09 00:00:01'

In [8]:
df.date_time.max()

'2016-05-11 23:59:59'

In [9]:
from datetime import datetime,date

In [10]:
%%time

df.date_time[0].split(' ')[0]
df['date'] = df.date_time.apply(lambda x: x.split(' ')[0])
print(df['date'].nunique())
print(df['date'].unique())

3
['2016-05-09' '2016-05-10' '2016-05-11']
Wall time: 1.08 s


### Drop unnecessary rows

In [11]:
%%time

# reset index to make sure that index values are unique
df = df.reset_index(drop=True)

# drop rows where exclude_hit > 1
df = df.drop(df[df.exclude_hit > 0].index)

# drop rows where hit_source is 5, 7, 8 or 9
df = df.drop(df[(df.hit_source == 5) | (df.hit_source == 7) | (df.hit_source == 8) | (df.hit_source == 9)].index)

Wall time: 7.24 s


### Standard mapping

In [12]:
%%time

### country mapping

# reset index to make sure that index values are unique
df = df.reset_index(drop=True)

# load file for country mapping
country_mapping = pd.read_csv('mapping_files/country.tsv', sep='\t', header=None)
country_mapping.columns = ['country_id', 'country_name']

# drop dupliate countries
country_mapping = country_mapping.drop_duplicates('country_name').reset_index(drop=True)

# create dictionary for country mapping
country_mapping_dict = dict(zip(country_mapping.country_id, country_mapping.country_name))

# map countries
df['country'] = df['country'].map(country_mapping_dict).fillna(df['country'])

Wall time: 2.18 s


In [13]:
%%time

### browser mapping

# reset index to make sure that index values are unique
df = df.reset_index(drop=True)

# load file for browser mapping
browser_mapping = pd.read_csv('mapping_files/browser_type.tsv', sep='\t', header=None)
browser_mapping.columns = ['browser_id', 'browser_name']

# create dictionary for browser mapping
browser_mapping_dict = dict(zip(browser_mapping.browser_id, browser_mapping.browser_name))

# map browsers
df['browser'] = df['browser'].map(browser_mapping_dict).fillna(df['browser'])
df['browser'] = df['browser'].apply(lambda x: 'Not Specified' if x == 0 else x)

Wall time: 5.71 s


In [14]:
%%time

### operating system mapping

# reset index to make sure that index values are unique
df = df.reset_index(drop=True)

# load file for operating system mapping
operating_system_mapping = pd.read_csv('mapping_files/operating_systems.tsv', sep='\t', header=None)
operating_system_mapping.columns = ['operating_system_id', 'operating_system_name']

# create dictionary for operating system mapping
operating_system_mapping_dict = dict(zip(operating_system_mapping.operating_system_id, operating_system_mapping.operating_system_name))

# map operating systems
df['operating_system'] = df['os'].map(operating_system_mapping_dict).fillna(df['os'])
df.drop('os', axis=1, inplace=True)

# generalize operating system
def generalize_operating_system(row):
    if 'Windows' in row['operating_system']:
        return 'Windows'
    elif 'Linux' in row['operating_system']:
        return 'Linux'
    elif 'Android' in row['operating_system']:
        return 'Android'
    elif 'Mobile iOS' in row['operating_system']:
        return 'Apple'
    elif 'Macintosh' in row['operating_system']:
        return 'Apple'
    elif 'OS X' in row['operating_system']:
        return 'Apple'
    elif 'Not Specified' in row['operating_system']:
        return 'Not Specified'
    else:
        return 'Other'
    
df['operating_system_generalized'] = df.apply(generalize_operating_system, axis=1)

Wall time: 1min 14s


In [15]:
%%time

### search engine mapping

# reset index to make sure that index values are unique
df = df.reset_index(drop=True)

# load file for search engine mapping
search_engine_mapping = pd.read_csv('mapping_files/search_engines.tsv', sep='\t', header=None)
search_engine_mapping.columns = ['search_engine_id', 'search_engine_name']

# create dictionary for search engine mapping
search_engine_mapping_dict = dict(zip(search_engine_mapping.search_engine_id, search_engine_mapping.search_engine_name))

# map search engines
df['search_engine'] = df['post_search_engine'].map(search_engine_mapping_dict).fillna(df['post_search_engine'])
df.drop('post_search_engine', axis=1, inplace=True)

# convert search engine column to string type
df['search_engine'] = df['search_engine'].astype(str)

# generalize search engine
def generalize_search_engine(row):
    if 'Google' in row['search_engine']:
        return 'Google'
    elif 'Yahoo' in row['search_engine']:
        return 'Yahoo'
    elif 'Bing' in row['search_engine']:
        return 'Bing'
    elif 'Baidu' in row['search_engine']:
        return 'Baidu'
    elif 'DuckDuckGo' in row['search_engine']:
        return 'DuckDuckGo'
    elif 'Yandex' in row['search_engine']:
        return 'Yandex'
    elif 'Search.ch' in row['search_engine']:
        return 'Search.ch'
    elif '0' in row['search_engine']:
        return ' Not Specified'
    else:
        return 'Other'
    
df['search_engine_generalized'] = df.apply(generalize_search_engine, axis=1)

Wall time: 2min 22s


In [16]:
%%time

### connection type mapping

# reset index to make sure that index values are unique
df = df.reset_index(drop=True)

# load file for connection type mapping
connection_type_mapping = pd.read_csv('mapping_files/connection_type.tsv', sep='\t', header=None)
connection_type_mapping.columns = ['connection_type_id', 'connection_type_name']

# create dictionary for connection type mapping
connection_type_mapping_dict = dict(zip(connection_type_mapping.connection_type_id, connection_type_mapping.connection_type_name))

# map connection types
df['connection_type'] = df['connection_type'].map(connection_type_mapping_dict).fillna(df['connection_type'])

Wall time: 4.92 s


In [17]:
%%time

### referrer type mapping

# reset index to make sure that index values are unique
df = df.reset_index(drop=True)

# load file for referrer type mapping
referrer_type_mapping = pd.read_csv('mapping_files/referrer_type.tsv', sep='\t', header=None)
referrer_type_mapping.columns = ['referrer_type_id', 'referrer_type_name', 'referrer_type']

# create dictionary for referrer type mapping
referrer_type_mapping_dict = dict(zip(referrer_type_mapping.referrer_type_id, referrer_type_mapping.referrer_type))

# map referrer types
df['referrer_type'] = df['ref_type'].map(referrer_type_mapping_dict).fillna(df['ref_type'])
df.drop('ref_type', axis=1, inplace=True)

Wall time: 7.88 s


### Custom mapping

In [18]:
%%time

### marketing channel mapping

# reset index to make sure that index values are unique
df = df.reset_index(drop=True)

# load file for marketing channel mapping
marketing_channel_mapping = pd.read_csv('mapping_files/siroop_marketingchannels_181009.tsv', sep='\t')

# create dictionary for marketing channel mapping
marketing_channel_mapping_dict = dict(zip(marketing_channel_mapping.channel_id, marketing_channel_mapping.name))

# map marketing channels
df['marketing_channel'] = df['va_closer_id'].map(marketing_channel_mapping_dict).fillna(df['va_closer_id'])
df['marketing_channel'] = df['marketing_channel'].apply(lambda x: 'Not Specified' if x == 0 else x)
df.drop('va_closer_id', axis=1, inplace=True)

Wall time: 6.06 s


In [19]:
%%time

### custom evars mapping

# reset index to make sure that index values are unique
df = df.reset_index(drop=True)

evars = pd.read_csv('mapping_files/siroop_evars_181009.tsv', sep='\t')
evars_mapping = evars[['id', 'name']]

evar_cols = [x for x in df.columns if x.lower()[:9] == 'post_evar']
evar_cols = [x.replace('post_', '') for x in evar_cols]
evars_mapped = evars[evars['id'].isin(evar_cols)][['id', 'name']]
evars_mapped['id'] = evars_mapped['id'].apply(lambda x: 'post_' + x)
evars_mapped = evars_mapped.reset_index(drop=True)

# rename evar columns
for i in range(evars_mapped.shape[0]):
    df.rename(columns={evars_mapped.iloc[i,0] : str.lower(evars_mapped.iloc[i,1]).replace(' ','_')}, inplace=True)

Wall time: 32 s


In [20]:
%%time

### custom props mapping

# reset index to make sure that index values are unique
df = df.reset_index(drop=True)

props = pd.read_csv('mapping_files/siroop_props_181009.tsv', sep='\t')
props_mapping = props[['id', 'name']]

prop_cols = [x for x in df.columns if x.lower()[:9] == 'post_prop']
prop_cols = [x.replace('post_', '') for x in prop_cols]
props_mapped = props[props['id'].isin(prop_cols)][['id', 'name']]
props_mapped['id'] = props_mapped['id'].apply(lambda x: 'post_' + x)
props_mapped = props_mapped.reset_index(drop=True)

# rename prop columns
for i in range(props_mapped.shape[0]):
    df.rename(columns={props_mapped.iloc[i,0] : str.lower(props_mapped.iloc[i,1]).replace(' ','_')}, inplace=True)

Wall time: 7.02 s


In [21]:
# fill missing values in post_event_list

df['post_event_list'] = df['post_event_list'].fillna('Not Specified')

In [22]:
%%time

### events mapping

# reset index to make sure that index values are unique
df = df.reset_index(drop=True)

# load file for event mapping
standard_events = pd.read_csv('mapping_files/event.tsv', sep='\t', header=None)
standard_events.columns = ['event_id', 'event_name']

# load file for custom event mapping
custom_events = pd.read_csv('mapping_files/siroop_events_181009.tsv', sep='\t')
custom_events['event_id'] = custom_events.index + 200

events = pd.merge(standard_events, custom_events, how='inner', on='event_id')
events_mapping = events[['event_id', 'name']]
events_mapping = events_mapping.reset_index(drop=True)

# iterate through event list and create dummies
for id, event in zip(events_mapping.iloc[:,0], events_mapping.iloc[:,1]):
        df[str.lower(event).replace(' ','_')] = df['post_event_list'].apply(lambda x: 1 if ','+str(id)+',' in x else 0)

Wall time: 2min 45s


In [23]:
%%time

# drop internal users
df = df.drop(df[df['internal_user_(e30)'] == 1].index)

Wall time: 11.8 s


### Aggregate data to visitor level

In [24]:
%%time

df['visitor_id'].nunique()

Wall time: 300 ms


188746

In [25]:
%%time

df.isnull().sum().sort_values(ascending=False).head(50)

Wall time: 11.6 s


user_server                                        1193133
duplicate_events                                   1192808
transaction_id_(v40)                               1191481
payment_type_of_completed_order_(v38)              1191481
post_purchaseid                                    1191436
post_transactionid                                 1191436
ct_connect_type                                    1189186
internal_search_-_clicked_result_position_(p13)    1188838
net_promoter_score_raw_(v10)_-_user                1185906
reg._user_id_(user)_(v36)                          1167660
registered_user_(user)_(v34)                       1167660
user_gender_(v61)                                  1160866
user_zip_code_(v60)                                1154624
user_age_(v62)                                     1154155
event/component_sub-category1_(v4)                 1152751
cart_value_(v50)                                   1114562
post_tnt_action                                    11135

In [26]:
%%time

# get na only and static columns
na_only_columns = []
static_columns = []

for i in df.columns:
    n_unique_values = df[str(i)].nunique()
    if n_unique_values == 0:
        na_only_columns.append(i)
    elif n_unique_values == 1:
        static_columns.append(i)
    else:
        pass

Wall time: 14.2 s


In [27]:
na_only_columns

['user_server']

In [28]:
sorted(static_columns)

['abstract_or_concrete_product_detail_view_(e60)',
 'abstract_prod_detail_view_(e57)_-_once_per_page',
 'active_merchant_skus_(e54)',
 'ad_cost_(e37)',
 'audiencestream_returning_visitor_(e28)',
 'cancelled_orders_(e45)',
 'cancelled_revenue_(e44)',
 'cancelled_units_(e46)',
 'cart_add_from_wishlist_(e83)',
 'cart_open_boolean',
 'checkout_step:_overview_(e4)',
 'checkout_step:_payment_(e3)',
 'clicks_(e36)',
 'concrete_prod_detail_view_(e56)_-_once_per_pageview',
 'concrete_prod_detail_view_(e59)',
 'curr_factor',
 'curr_rate',
 'custom_301',
 'custom_302',
 'custom_303',
 'custom_304',
 'custom_5',
 'custom_69',
 'custom_76',
 'custom_79',
 'custom_84',
 'custom_88',
 'custom_89',
 'custom_90',
 'custom_91',
 'custom_92',
 'custom_93',
 'custom_94',
 'data_source_import_event_(e100)',
 'delivered',
 'duplicate_events',
 'exclude_hit',
 'fast_checkout_(e6)',
 'hit_of_registered_user_(e24)_-_notyet_active',
 'hit_source',
 'hit_with_audiencestream_dle_(e47)',
 'hit_with_third-party_coo

#### Numerical columns

In [30]:
from datetime import datetime, date

In [31]:
%%time

# convert hit_time_gmt column from unix format to datetime format
df['hit_time_gmt'] = pd.to_datetime(df['hit_time_gmt'], unit='s')

Wall time: 1.11 s


In [32]:
# convert visit_page_num column to integer type
df['visit_page_num'] = df['visit_page_num'].astype(np.int64)

In [33]:
# fill missing values with 0 in cart_value_(v50) column
df['cart_value_(v50)'].fillna(0, inplace=True)

In [85]:
%%time

# select numerical columns for aggregation
#agg_cols = ['visitor_id', 'visit_num', 'visit_page_num', 'hit_time_gmt' ,'purchase_boolean', 'product_view_boolean', 
#            'cart_open_boolean', 'checkout_boolean', 'cart_addition_boolean', 'cart_removal_boolean', 'cart_view_boolean', 
#            'campaign_view_boolean', 'cart_value_(v50)', 'page_view_boolean', 'last_purchase_num']
agg_cols = ['visitor_id', 'visit_num', 'visit_page_num', 'hit_time_gmt' ,'purchase_boolean', 'product_view_boolean', 
            'checkout_boolean', 'cart_addition_boolean', 'cart_removal_boolean', 'cart_view_boolean',
            'campaign_view_boolean', 'cart_value_(v50)', 'page_view_boolean', 'last_purchase_num']

agg_df = df.loc[:, df.columns.isin(agg_cols)].copy()

# note: cart_open_boolean has only one unique value == 0
# what about sum of price of products viewed?

Wall time: 435 ms


In [86]:
%%time

# group by visitor_id and visit_num and aggregate numerical columns
agg_df = agg_df.groupby(by = ['visitor_id', 'visit_num'], as_index=False).agg({'visit_page_num' : 'max',
                                                                             'hit_time_gmt': ['min', 'max'],
                                                                             'purchase_boolean' : 'sum',
                                                                             'product_view_boolean' : 'sum',
                                                                             'checkout_boolean' : 'sum',
                                                                             'cart_addition_boolean': 'sum',
                                                                             'cart_removal_boolean': 'sum',
                                                                             'cart_view_boolean': 'sum',
                                                                             'campaign_view_boolean': 'sum',
                                                                             'cart_value_(v50)': 'sum',
                                                                             'page_view_boolean': 'sum',
                                                                             'last_purchase_num': 'max'
                                                                            })
agg_df = agg_df.reset_index(drop=True)
agg_df.columns = list(agg_df.columns)
agg_df.columns = ['visitor_id', 'visit_num', 'visit_page_num', 'hit_time_gmt', 'last_hit_time_gmt_visit', 'purchase',
                  'product_view', 'checkout', 'cart_addition', 'cart_removal', 'cart_view', 'campaign_view', 'cart_value', 
                  'page_view', 'last_purchase_num']

Wall time: 5.36 s


In [87]:
%%time

# sort dataframe by visitor_id, visit_num and hit_time_gmt_min
agg_df = agg_df.sort_values(['visitor_id', 'visit_num', 'hit_time_gmt', 'last_hit_time_gmt_visit'], ascending=[True, True, True, True])

Wall time: 631 ms


In [88]:
agg_df.shape

(214009, 15)

In [89]:
agg_df.head()

,visitor_id,visit_num,visit_page_num,hit_time_gmt,last_hit_time_gmt_visit,purchase,product_view,checkout,cart_addition,cart_removal,cart_view,campaign_view,cart_value,page_view,last_purchase_num
0,1000003101246344307_5750083886418766345,1,10,2016-05-11 11:46:56,2016-05-11 12:06:06,0,3,0,1,0,0,3,0.0,7,0
1,100001127200644523_1523571685696238827,1,3,2016-05-11 19:34:38,2016-05-11 19:34:59,0,1,0,0,0,0,1,0.0,2,0
2,1000122849524883538_568006005550457493,1,1,2016-05-10 17:58:31,2016-05-10 17:58:31,0,1,0,0,0,0,1,0.0,1,0
3,1000137032849919908_4977555635345216400,1,1,2016-05-09 09:00:03,2016-05-09 09:00:03,0,1,0,0,0,0,1,0.0,1,0
4,1000156419831429093_6698187625843757298,1,7,2016-05-10 17:36:58,2016-05-10 17:37:43,0,1,0,0,0,0,2,0.0,4,0


In [117]:
agg_df['visitor_id'].nunique()

188746

#### Categorical columns

In [148]:
%%time

# select categorical columns
categorical_cols = ['visitor_id', 'visit_num', 'hit_time_gmt', 'connection_type', 'country', 'geo_city', 'geo_dma', 
                    'geo_region', 'geo_zip', 'post_channel', 'post_cookies', 'net_promoter_score_raw_(v10)_-_user', 
                    'registered_user_(user)_(v34)', 'login_status_(hit)_(v37)', 'user_gender_(v61)', 'user_age_(v62)',
                    'post_persistent_cookie', 'browser_generalized', 'operating_system_generalized',
                    'search_engine_generalized', 'marketing_channel', 'referrer_type', 'hit_of_logged_in_user_(e23)',
                    'visit_during_tv_spot_(e71)', 'repeat_orders_(e9)', 'newsletter_subscriber_(e27)', 
                    'registration_(any_form)_(e20)', 'registration_fail_(e75)', 'newsletter_signup_(any_form)_(e26)', 
                    'new_visit', 'hourly_visitor', 'daily_visitor', 'weekly_visitor', 'monthly_visitor', 
                    'quarterly_visitor', 'yearly_visitor']

categorical_df = df.loc[:, df.columns.isin(categorical_cols)].copy()

Wall time: 751 ms


In [149]:
# for now: drop columns with many missing values and/or unclear usefulness
categorical_df.drop(['country',
                     'geo_city', 
                     'geo_dma',
                     'geo_region', 
                     'geo_zip',
                     'net_promoter_score_raw_(v10)_-_user', 
                     'user_gender_(v61)', 
                     'user_age_(v62)',
                     'newsletter_subscriber_(e27)',
                     'registration_(any_form)_(e20)',
                     'registration_fail_(e75)',
                     'newsletter_signup_(any_form)_(e26)',
                     'hourly_visitor', 
                     'daily_visitor', 
                     'weekly_visitor', 
                     'monthly_visitor', 
                     'quarterly_visitor', 
                     'yearly_visitor'], axis=1, inplace=True)

# how to handle geographic information?

In [150]:
categorical_df['post_cookies'] = categorical_df['post_cookies'].apply(lambda x: 1 if x == 'Y' else 0)

In [151]:
categorical_df['registered_user'] = categorical_df['registered_user_(user)_(v34)'].apply(lambda x: 1 if x == 'y' else 0)
categorical_df.drop('registered_user_(user)_(v34)', axis=1, inplace=True)

In [152]:
categorical_df['login_status'] = categorical_df['login_status_(hit)_(v37)'].apply(lambda x: 1 if x == 'y' else 0)
categorical_df.drop('login_status_(hit)_(v37)', axis=1, inplace=True)

In [153]:
categorical_df['post_persistent_cookie'] = categorical_df['post_persistent_cookie'].apply(lambda x: 1 if x == 'Y' else 0)

In [154]:
categorical_df['post_channel'] = df['post_channel'].fillna('Not Specified')

In [155]:
#categorical_df['geo_city'] = categorical_df['geo_city'].apply(lambda x: 'Not Specified' if x == '?' else x)
#categorical_df['geo_region'] = categorical_df['geo_region'].apply(lambda x: 'Not Specified' if x == '?' else x)
#categorical_df['geo_region'] = df['geo_region'].fillna('Not Specified')

In [156]:
categorical_df.shape

(1193133, 17)

In [157]:
categorical_df.head()

,connection_type,hit_time_gmt,new_visit,post_channel,post_cookies,post_persistent_cookie,visit_num,visitor_id,operating_system_generalized,search_engine_generalized,referrer_type,marketing_channel,repeat_orders_(e9),hit_of_logged_in_user_(e23),visit_during_tv_spot_(e71),registered_user,login_status
0,Mobile Carrier,2016-05-08 22:01:05,1,Search Results,1,1,1,469902274971074530_2331856232162590288,Apple,Not Specified,bookmarked,Paid Search Other,0,0,0,0,0
1,LAN/Wifi,2016-05-08 22:01:05,1,Search Results,1,1,1,555094004133467786_3187591169842925334,Windows,Google,search_engine,Paid Search Other,0,0,0,0,0
2,LAN/Wifi,2016-05-08 22:00:54,1,Homepage,1,1,1,3837330188914495230_2270870067931637094,Apple,Not Specified,bookmarked,Display Prospecting,0,0,0,0,0
3,Mobile Carrier,2016-05-08 22:00:17,1,Search Results,1,1,1,7852916370904039970_29230011524970538,Apple,Not Specified,bookmarked,Paid Search Other,0,0,0,0,0
4,LAN/Wifi,2016-05-08 22:00:42,1,Search Results,1,1,1,4163393833230664452_5491479092785268651,Apple,Google,search_engine,Paid Search Other,0,0,0,0,0


In [158]:
categorical_df['visitor_id'].nunique()

188746

#### Merge numerical and categorical columns

In [159]:
%%time

df_visitor_level = agg_df.merge(categorical_df, on=['visitor_id', 'visit_num', 'hit_time_gmt'], how='left')

Wall time: 1.28 s


In [160]:
df_visitor_level.shape

(217586, 29)

#### Duplicated rows after merge?

In [108]:
df_visitor_level[df_visitor_level.duplicated(keep=False)]

,visitor_id,visit_num,visit_page_num,hit_time_gmt,last_hit_time_gmt_visit,purchase,product_view,checkout,cart_addition,cart_removal,cart_view,campaign_view,cart_value,page_view,last_purchase_num,connection_type,geo_city,new_visit,post_channel,post_cookies,post_persistent_cookie,operating_system_generalized,search_engine_generalized,referrer_type,marketing_channel,repeat_orders_(e9),hit_of_logged_in_user_(e23),visit_during_tv_spot_(e71),registered_user,login_status
26349,2009292567374421351_4190843545824568702,1,3,2016-05-11 17:36:28,2016-05-11 17:36:28,0,0,0,0,0,0,3,0.00,3,0,LAN/Wifi,la tour-de-peilz,0,Homepage,1,1,Android,Not Specified,internal,DBM Prospecting,0,0,0,0,0
26350,2009292567374421351_4190843545824568702,1,3,2016-05-11 17:36:28,2016-05-11 17:36:28,0,0,0,0,0,0,3,0.00,3,0,LAN/Wifi,la tour-de-peilz,0,Homepage,1,1,Android,Not Specified,internal,DBM Prospecting,0,0,0,0,0
86620,4298597618857702079_7046983543175596625,1,3,2016-05-11 11:29:18,2016-05-11 11:29:18,0,0,0,0,0,0,1,0.00,1,0,LAN/Wifi,zurich,0,Search Results,0,1,Apple,Not Specified,bookmarked,Paid Search Other,0,0,0,0,0
86621,4298597618857702079_7046983543175596625,1,3,2016-05-11 11:29:18,2016-05-11 11:29:18,0,0,0,0,0,0,1,0.00,1,0,LAN/Wifi,zurich,0,Search Results,0,1,Apple,Not Specified,bookmarked,Paid Search Other,0,0,0,0,0
131360,6015328801725495790_9163797718063998433,158,22,2016-05-09 08:40:08,2016-05-09 09:50:47,0,0,0,0,0,0,0,854.05,16,2,Mobile Carrier,zurich,0,Corporate,1,1,Apple,Not Specified,internal,Referring Domains,0,1,0,1,1
131361,6015328801725495790_9163797718063998433,158,22,2016-05-09 08:40:08,2016-05-09 09:50:47,0,0,0,0,0,0,0,854.05,16,2,Mobile Carrier,zurich,0,Corporate,1,1,Apple,Not Specified,internal,Referring Domains,0,1,0,1,1


In [54]:
%%time

#df_visitor_level = df_visitor_level.drop_duplicates(subset=['visitor_id', 'visit_num', 'hit_time_gmt', 'last_hit_time_gmt_visit'], keep=False)

Wall time: 0 ns


### Add features and preprocess data for modeling

In [126]:
df_visitor_level.head()

,visitor_id,visit_num,visit_page_num,hit_time_gmt,last_hit_time_gmt_visit,purchase,product_view,checkout,cart_addition,cart_removal,cart_view,campaign_view,cart_value,page_view,last_purchase_num,connection_type,geo_city,new_visit,post_channel,post_cookies,post_persistent_cookie,operating_system_generalized,search_engine_generalized,referrer_type,marketing_channel,repeat_orders_(e9),hit_of_logged_in_user_(e23),visit_during_tv_spot_(e71),registered_user,login_status,bounce,day_of_week,hour_of_day,visit_duration_seconds
0,1000003101246344307_5750083886418766345,1,10,2016-05-11 11:46:56,2016-05-11 12:06:06,0,3,0,1,0,0,3,0.0,7,0,LAN/Wifi,ebikon,1,Product,1,1,Apple,Google,search_engine,Paid Search G. Shopping,0,0,0,0,0,0,2,11,1150
1,100001127200644523_1523571685696238827,1,3,2016-05-11 19:34:38,2016-05-11 19:34:59,0,1,0,0,0,0,1,0.0,2,0,Mobile Carrier,zurich,1,Search Results,1,1,Android,Not Specified,social_network,Social Media Prospecting,0,0,0,0,0,0,2,19,21
2,1000122849524883538_568006005550457493,1,1,2016-05-10 17:58:31,2016-05-10 17:58:31,0,1,0,0,0,0,1,0.0,1,0,LAN/Wifi,zollikon,1,Product,1,1,Apple,Google,search_engine,Paid Search G. Shopping,0,0,1,0,0,1,1,17,0
3,1000137032849919908_4977555635345216400,1,1,2016-05-09 09:00:03,2016-05-09 09:00:03,0,1,0,0,0,0,1,0.0,1,0,LAN/Wifi,einsiedeln,1,Product,1,1,Windows,Google,search_engine,Paid Search Other,0,0,0,0,0,1,0,9,0
4,1000156419831429093_6698187625843757298,1,7,2016-05-10 17:36:58,2016-05-10 17:37:43,0,1,0,0,0,0,2,0.0,4,0,Mobile Carrier,zurich,1,Search Results,1,1,Apple,Google,search_engine,Paid Search Other,0,0,0,0,0,0,1,17,45


In [161]:
# add flag to indicate bounce
df_visitor_level['bounce'] = df_visitor_level['visit_page_num'].apply(lambda x: 1 if x == 1 else 0)

#### Encode target

In [162]:
%%time

# binary encode target to be either 1 or 0
df_visitor_level['purchase'] = df_visitor_level['purchase'].apply(lambda x: 1 if x >= 1 else 0)

Wall time: 135 ms


In [163]:
df_visitor_level['purchase'].value_counts()

0    215933
1      1653
Name: purchase, dtype: int64

In [164]:
1653/(215933+1635)

0.007597624650683924

#### Add time features

In [146]:
df_visitor_level.head()

,visitor_id,visit_num,visit_page_num,hit_time_gmt,last_hit_time_gmt_visit,purchase,product_view,checkout,cart_addition,cart_removal,cart_view,campaign_view,cart_value,page_view,last_purchase_num,connection_type,geo_city,new_visit,post_channel,post_cookies,post_persistent_cookie,operating_system_generalized,search_engine_generalized,referrer_type,marketing_channel,repeat_orders_(e9),hit_of_logged_in_user_(e23),visit_during_tv_spot_(e71),registered_user,login_status,bounce,day_of_week,hour_of_day,visit_duration_seconds,visitor_id_lag,last_hit_time_gmt_visit_lag,days_since_last_visit,visit_in_last_7_days
0,1000003101246344307_5750083886418766345,1,10,2016-05-11 11:46:56,2016-05-11 12:06:06,0,3,0,1,0,0,3,0.0,7,0,LAN/Wifi,ebikon,1,Product,1,1,Apple,Google,search_engine,Paid Search G. Shopping,0,0,0,0,0,0,2,11,1150,NaN,NaT,NaN,0
1,100001127200644523_1523571685696238827,1,3,2016-05-11 19:34:38,2016-05-11 19:34:59,0,1,0,0,0,0,1,0.0,2,0,Mobile Carrier,zurich,1,Search Results,1,1,Android,Not Specified,social_network,Social Media Prospecting,0,0,0,0,0,0,2,19,21,1000003101246344307_5750083886418766345,2016-05-11 12:06:06,NaN,0
2,1000122849524883538_568006005550457493,1,1,2016-05-10 17:58:31,2016-05-10 17:58:31,0,1,0,0,0,0,1,0.0,1,0,LAN/Wifi,zollikon,1,Product,1,1,Apple,Google,search_engine,Paid Search G. Shopping,0,0,1,0,0,1,1,17,0,100001127200644523_1523571685696238827,2016-05-11 19:34:59,NaN,0
3,1000137032849919908_4977555635345216400,1,1,2016-05-09 09:00:03,2016-05-09 09:00:03,0,1,0,0,0,0,1,0.0,1,0,LAN/Wifi,einsiedeln,1,Product,1,1,Windows,Google,search_engine,Paid Search Other,0,0,0,0,0,1,0,9,0,1000122849524883538_568006005550457493,2016-05-10 17:58:31,NaN,0
4,1000156419831429093_6698187625843757298,1,7,2016-05-10 17:36:58,2016-05-10 17:37:43,0,1,0,0,0,0,2,0.0,4,0,Mobile Carrier,zurich,1,Search Results,1,1,Apple,Google,search_engine,Paid Search Other,0,0,0,0,0,0,1,17,45,1000137032849919908_4977555635345216400,2016-05-09 09:00:03,NaN,0


In [142]:
df_visitor_level[df_visitor_level['days_since_last_visit'] == -1][['visitor_id', 'visitor_id_lag', 'hit_time_gmt',
                                                                   'last_hit_time_gmt_visit', 'last_hit_time_gmt_visit_lag']]

,visitor_id,visitor_id_lag,hit_time_gmt,last_hit_time_gmt_visit,last_hit_time_gmt_visit_lag
14,1000454914728368831_4602302257341836008,1000454914728368831_4602302257341836008,2016-05-09 15:04:14,2016-05-09 15:04:37,2016-05-09 15:04:37
121,1004228797914345198_4903034164002497409,1004228797914345198_4903034164002497409,2016-05-10 05:49:28,2016-05-10 05:59:08,2016-05-10 05:59:08
606,1022158454932198323_2252243808600020876,1022158454932198323_2252243808600020876,2016-05-09 20:20:24,2016-05-09 20:20:59,2016-05-09 20:20:59
609,1022206338657832982_7049096064805428480,1022206338657832982_7049096064805428480,2016-05-10 19:14:54,2016-05-10 19:16:14,2016-05-10 19:16:14
814,1028810289576812037_5450875944594665172,1028810289576812037_5450875944594665172,2016-05-11 15:09:16,2016-05-11 15:10:03,2016-05-11 15:10:03
957,1034020922144954511_6334310410586044151,1034020922144954511_6334310410586044151,2016-05-09 07:59:42,2016-05-09 08:00:09,2016-05-09 08:00:09
1033,1036770569768675356_4836739697730200887,1036770569768675356_4836739697730200887,2016-05-09 06:45:10,2016-05-09 06:45:32,2016-05-09 06:45:32
1133,1040702021714555309_2291076778316985846,1040702021714555309_2291076778316985846,2016-05-09 08:00:05,2016-05-09 08:01:15,2016-05-09 08:01:15
1186,1043251484920914474_7492279665307591034,1043251484920914474_7492279665307591034,2016-05-10 05:37:43,2016-05-10 05:44:27,2016-05-10 05:44:27
1208,1043811659135305420_8200770469326260445,1043811659135305420_8200770469326260445,2016-05-11 18:55:27,2016-05-11 18:55:34,2016-05-11 18:55:34


In [143]:
df_visitor_level['days_since_last_visit'].unique()

array([nan,  0., -1.,  1.,  2.])

In [144]:
df_visitor_level['days_since_last_visit'].value_counts()

 0.0    23014
 1.0     3503
-1.0     1485
 2.0      838
Name: days_since_last_visit, dtype: int64

In [165]:
%%time

# sort dataframe by visitor_id, visit_num and hit_time_gmt_min
df_visitor_level = df_visitor_level.sort_values(['visitor_id', 'visit_num', 'hit_time_gmt', 'last_hit_time_gmt_visit'], ascending=[True, True, True, True])

Wall time: 787 ms


In [166]:
# day of week
df_visitor_level['day_of_week'] = df_visitor_level['hit_time_gmt'].dt.dayofweek

In [167]:
# hour of day
df_visitor_level['hour_of_day'] = df_visitor_level['hit_time_gmt'].dt.hour

In [168]:
%%time

# calculate visit duration in seconds
df_visitor_level['visit_duration_seconds'] = df_visitor_level['last_hit_time_gmt_visit'] - df_visitor_level['hit_time_gmt']
df_visitor_level['visit_duration_seconds'] = df_visitor_level['visit_duration_seconds'].apply(lambda x: x.seconds)

Wall time: 3.45 s


In [169]:
%%time

# add lag columns for visitor_id and last_hit_time_gmt_visit
df_visitor_level['visitor_id_lag'] = df_visitor_level['visitor_id'].shift(1)
df_visitor_level['last_hit_time_gmt_visit_lag'] = df_visitor_level['last_hit_time_gmt_visit'].shift(1)

Wall time: 22.5 ms


In [170]:
%%time

# calculate days since last visit
df_visitor_level['days_since_last_visit'] = df_visitor_level.apply(lambda x: x['hit_time_gmt'] - x['last_hit_time_gmt_visit_lag'] 
                                       if x['visitor_id'] == x['visitor_id_lag'] 
                                       else np.nan, axis=1)
df_visitor_level['days_since_last_visit'] = df_visitor_level['days_since_last_visit'].apply(lambda x: x.days)

Wall time: 16.8 s


In [171]:
df_visitor_level['visit_in_last_7_days'] = df_visitor_level['days_since_last_visit'].apply(lambda x: 1 if (x >=1) & (x <= 7) else 0)

In [65]:
# days since last purchase

#### Create dummies for categorical columns

In [180]:
df_visitor_level.drop(['visitor_id', 
                       'visitor_id_lag', 
                       'last_hit_time_gmt_visit', 
                       'last_hit_time_gmt_visit_lag',
                       'days_since_last_visit'], axis=1, inplace=True)

In [173]:
object_cols = list(df_visitor_level.select_dtypes(include=['object']).columns)
object_cols

['connection_type',
 'post_channel',
 'operating_system_generalized',
 'search_engine_generalized',
 'referrer_type',
 'marketing_channel']

In [174]:
%%time

dummies = pd.get_dummies(df_visitor_level.loc[:, df_visitor_level.columns.isin(object_cols)], drop_first=True)

Wall time: 231 ms


In [177]:
df_visitor_level.drop(object_cols, axis=1, inplace=True)

In [178]:
%%time

df_visitor_level = pd.concat([df_visitor_level, dummies], axis=1)

Wall time: 62.4 ms


In [192]:
df_visitor_level.head()

,visit_num,visit_page_num,hit_time_gmt,purchase,product_view,checkout,cart_addition,cart_removal,cart_view,campaign_view,cart_value,page_view,last_purchase_num,new_visit,post_cookies,post_persistent_cookie,repeat_orders_(e9),hit_of_logged_in_user_(e23),visit_during_tv_spot_(e71),registered_user,login_status,bounce,day_of_week,hour_of_day,visit_duration_seconds,visit_in_last_7_days,connection_type_Mobile Carrier,connection_type_Modem,post_channel_Cart,post_channel_Category Page Level 1,post_channel_Category Page Level 2,post_channel_Category Page Level 3,post_channel_Checkout,post_channel_Corporate,post_channel_Homepage,post_channel_Information,post_channel_Jobs,post_channel_Merchants,post_channel_Not Specified,post_channel_Order Confirmation,post_channel_Other,post_channel_Product,post_channel_Search Results,post_channel_Themeworld,operating_system_generalized_Apple,operating_system_generalized_Linux,operating_system_generalized_Not Specified,operating_system_generalized_Other,operating_system_generalized_Windows,search_engine_generalized_Bing,search_engine_generalized_DuckDuckGo,search_engine_generalized_Google,search_engine_generalized_Other,search_engine_generalized_Search.ch,search_engine_generalized_Yahoo,search_engine_generalized_Yandex,referrer_type_external,referrer_type_internal,referrer_type_nojs,referrer_type_search_engine,referrer_type_social_network,marketing_channel_Digital Documents,marketing_channel_Direct,marketing_channel_Display Prospecting,marketing_channel_Editorial,marketing_channel_Mail Flaschenpost,marketing_channel_Mail Other,marketing_channel_Not Specified,marketing_channel_Organic Search,marketing_channel_Other Campaign,marketing_channel_Paid Search Brand,marketing_channel_Paid Search G. Shopping,marketing_channel_Paid Search Other,marketing_channel_Referring Domains,marketing_channel_Social Media Campaigns,marketing_channel_Social Media Non-Paid,marketing_channel_Social Media Prospecting
0,1,10,2016-05-11 11:46:56,0,3,0,1,0,0,3,0.0,7,0,1,1,1,0,0,0,0,0,0,2,11,1150,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,1,3,2016-05-11 19:34:38,0,1,0,0,0,0,1,0.0,2,0,1,1,1,0,0,0,0,0,0,2,19,21,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,1,1,2016-05-10 17:58:31,0,1,0,0,0,0,1,0.0,1,0,1,1,1,0,0,1,0,0,1,1,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,1,1,2016-05-09 09:00:03,0,1,0,0,0,0,1,0.0,1,0,1,1,1,0,0,0,0,0,1,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,1,7,2016-05-10 17:36:58,0,1,0,0,0,0,2,0.0,4,0,1,1,1,0,0,0,0,0,0,1,17,45,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


#### Train test split

In [198]:
y_train = df_visitor_level[df_visitor_level['hit_time_gmt'] <= '2016-05-10 23:59:59']['purchase']
X_train = df_visitor_level[df_visitor_level['hit_time_gmt'] <= '2016-05-10 23:59:59'].copy()
X_train.drop(['purchase', 'hit_time_gmt'], axis=1, inplace=True)

In [199]:
y_test = df_visitor_level[df_visitor_level['hit_time_gmt'] > '2016-05-10 23:59:59']['purchase']
X_test = df_visitor_level[df_visitor_level['hit_time_gmt'] > '2016-05-10 23:59:59'].copy()
X_test.drop(['purchase', 'hit_time_gmt'], axis=1, inplace=True)

In [200]:
y_train.head()

2    0
3    0
4    0
5    0
6    0
Name: purchase, dtype: int64

In [205]:
X_train.shape

(154829, 75)

In [201]:
X_train.head()

,visit_num,visit_page_num,product_view,checkout,cart_addition,cart_removal,cart_view,campaign_view,cart_value,page_view,last_purchase_num,new_visit,post_cookies,post_persistent_cookie,repeat_orders_(e9),hit_of_logged_in_user_(e23),visit_during_tv_spot_(e71),registered_user,login_status,bounce,day_of_week,hour_of_day,visit_duration_seconds,visit_in_last_7_days,connection_type_Mobile Carrier,connection_type_Modem,post_channel_Cart,post_channel_Category Page Level 1,post_channel_Category Page Level 2,post_channel_Category Page Level 3,post_channel_Checkout,post_channel_Corporate,post_channel_Homepage,post_channel_Information,post_channel_Jobs,post_channel_Merchants,post_channel_Not Specified,post_channel_Order Confirmation,post_channel_Other,post_channel_Product,post_channel_Search Results,post_channel_Themeworld,operating_system_generalized_Apple,operating_system_generalized_Linux,operating_system_generalized_Not Specified,operating_system_generalized_Other,operating_system_generalized_Windows,search_engine_generalized_Bing,search_engine_generalized_DuckDuckGo,search_engine_generalized_Google,search_engine_generalized_Other,search_engine_generalized_Search.ch,search_engine_generalized_Yahoo,search_engine_generalized_Yandex,referrer_type_external,referrer_type_internal,referrer_type_nojs,referrer_type_search_engine,referrer_type_social_network,marketing_channel_Digital Documents,marketing_channel_Direct,marketing_channel_Display Prospecting,marketing_channel_Editorial,marketing_channel_Mail Flaschenpost,marketing_channel_Mail Other,marketing_channel_Not Specified,marketing_channel_Organic Search,marketing_channel_Other Campaign,marketing_channel_Paid Search Brand,marketing_channel_Paid Search G. Shopping,marketing_channel_Paid Search Other,marketing_channel_Referring Domains,marketing_channel_Social Media Campaigns,marketing_channel_Social Media Non-Paid,marketing_channel_Social Media Prospecting
2,1,1,1,0,0,0,0,1,0.0,1,0,1,1,1,0,0,1,0,0,1,1,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,1,1,1,0,0,0,0,1,0.0,1,0,1,1,1,0,0,0,0,0,1,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,1,7,1,0,0,0,0,2,0.0,4,0,1,1,1,0,0,0,0,0,0,1,17,45,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
5,1,1,1,0,0,0,0,1,0.0,1,0,1,1,1,0,0,0,0,0,1,1,23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6,1,1,0,0,0,0,0,1,0.0,1,0,1,1,1,0,0,0,0,0,1,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [202]:
y_test.head()

0    0
1    0
7    0
8    0
9    0
Name: purchase, dtype: int64

In [204]:
X_test.shape

(62757, 75)

In [203]:
X_test.head()

,visit_num,visit_page_num,product_view,checkout,cart_addition,cart_removal,cart_view,campaign_view,cart_value,page_view,last_purchase_num,new_visit,post_cookies,post_persistent_cookie,repeat_orders_(e9),hit_of_logged_in_user_(e23),visit_during_tv_spot_(e71),registered_user,login_status,bounce,day_of_week,hour_of_day,visit_duration_seconds,visit_in_last_7_days,connection_type_Mobile Carrier,connection_type_Modem,post_channel_Cart,post_channel_Category Page Level 1,post_channel_Category Page Level 2,post_channel_Category Page Level 3,post_channel_Checkout,post_channel_Corporate,post_channel_Homepage,post_channel_Information,post_channel_Jobs,post_channel_Merchants,post_channel_Not Specified,post_channel_Order Confirmation,post_channel_Other,post_channel_Product,post_channel_Search Results,post_channel_Themeworld,operating_system_generalized_Apple,operating_system_generalized_Linux,operating_system_generalized_Not Specified,operating_system_generalized_Other,operating_system_generalized_Windows,search_engine_generalized_Bing,search_engine_generalized_DuckDuckGo,search_engine_generalized_Google,search_engine_generalized_Other,search_engine_generalized_Search.ch,search_engine_generalized_Yahoo,search_engine_generalized_Yandex,referrer_type_external,referrer_type_internal,referrer_type_nojs,referrer_type_search_engine,referrer_type_social_network,marketing_channel_Digital Documents,marketing_channel_Direct,marketing_channel_Display Prospecting,marketing_channel_Editorial,marketing_channel_Mail Flaschenpost,marketing_channel_Mail Other,marketing_channel_Not Specified,marketing_channel_Organic Search,marketing_channel_Other Campaign,marketing_channel_Paid Search Brand,marketing_channel_Paid Search G. Shopping,marketing_channel_Paid Search Other,marketing_channel_Referring Domains,marketing_channel_Social Media Campaigns,marketing_channel_Social Media Non-Paid,marketing_channel_Social Media Prospecting
0,1,10,3,0,1,0,0,3,0.0,7,0,1,1,1,0,0,0,0,0,0,2,11,1150,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,1,3,1,0,0,0,0,1,0.0,2,0,1,1,1,0,0,0,0,0,0,2,19,21,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
7,1,15,0,0,0,0,0,0,0.0,10,0,1,1,1,0,0,1,0,0,0,2,18,388,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,1,6,1,0,0,0,0,2,0.0,3,0,1,1,1,0,0,1,0,0,0,2,20,96,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
9,1,1,0,0,0,0,0,0,0.0,1,0,1,1,1,0,0,0,0,0,1,2,10,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


### Modeling

In [218]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import KFold,cross_val_score
from sklearn.metrics import accuracy_score,roc_curve,auc,confusion_matrix,classification_report

C:\Users\alex.merdian-tarko\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [207]:
seed = 0
scoring = 'accuracy'

In [ ]:
%%time

models = []
models.append(('LR', LogisticRegression()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('SVM', SVC()))
models.append(('RF', RandomForestClassifier()))

results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=10, random_state=seed)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = '%s: %f (%f)' % (name, cv_results.mean(), cv_results.std())
    print(msg)

LR: 0.995795 (0.000688)
CART: 0.996919 (0.000409)
NB: 0.965123 (0.002477)
KNN: 0.992030 (0.000508)


#### Logistic Regression

In [208]:
lr = LogisticRegression()

In [211]:
%%time

lr.fit(X_train, y_train)

Wall time: 9.6 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [212]:
%%time

y_pred = model.predict(X_test)

Wall time: 56.9 ms


In [215]:
%%time

accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: %.2f%%' % (accuracy * 100.0))
print('\n')
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
print('AUC: %.2f' % auc(fpr, tpr))
print('\n')
print('Confusion matrix')
print(confusion_matrix(y_test, y_pred))
print('\n')
print('Classification report')
print(classification_report(y_test, y_pred))

Accuracy: 99.47%


AUC: 0.75


Confusion matrix
[[62142    64]
 [  271   280]]


Classification report
             precision    recall  f1-score   support

          0       1.00      1.00      1.00     62206
          1       0.81      0.51      0.63       551

avg / total       0.99      0.99      0.99     62757

Wall time: 85.9 ms


true negatives C[0,0]
false negatives C[1,0]
true positives C[1,1]
false positives is C[0,1]